In [ ]:

import numpy as np
import random
import pandas as pd


In [ ]:
!pip install pyreadr
# import pyreader
try:
    import google.colab
    # Running on Google Colab, so install Biopython first
    !pip install biopython
except ImportError:
    pass
import os
import sys

from urllib.request import urlretrieve
from Bio import SeqIO, SearchIO, Entrez
from Bio.Seq import Seq
from Bio.SeqUtils import GC
from Bio.Blast import NCBIWWW
from Bio.Data import CodonTable

print("Python version:", sys.version_info)
import numpy as np

import gzip
from mimetypes import guess_type
from functools import partial
import pyreadr


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.8/434.8 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 7.0 MB/s eta 0:00:00
Python version: sys.version_info(major=3, minor=10, micro=12, releaselevel='final', serial=0)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
processing = True
# change to true if you start the pre-process from scratch
if processing:
  raw_data_brca_path = "/content/drive/MyDrive/MSC/CH3_Project/Spatial Methyl/raw_data/BRCA.RData"
  raw_data_luad_path = "/content/drive/MyDrive/MSC/CH3_Project/Spatial Methyl/raw_data/LUAD.RData"
  probe_to_serrounding_seq_path = "/content/drive/MyDrive/MSC/CH3_Project/res/hg_chromosoms/probeToSurroundingSeqFilePrefixAll/probe_to_surroundingSeq.csv"
  probe_to_serrounding_seq_one_hot_path= "/content/drive/MyDrive/MSC/CH3_Project/res/probe_to_surrounding_seq_one_hot.csv"

  raw_data_brca = pyreadr.read_r(raw_data_brca_path)
  raw_data_luad = pyreadr.read_r(raw_data_luad_path)

  methyl_brca = raw_data_brca['methyl']
  methyl_luad = raw_data_luad['methyl']
  expression_brca = raw_data_brca['expressi']
  expression_luad = raw_data_luad['expressi']

  print(f"methyl_brca shape is {methyl_brca.shape}")
  print(methyl_brca.head())
  print(f"expression_brca shape is {expression_brca.shape}")
  print(expression_brca.head())
  print(f"methyl_luad shape is {methyl_luad.shape}")
  print(methyl_luad.head())
  print(f"expression_luad shape is {expression_luad.shape}")
  print(expression_luad.head())


In [ ]:
preprocess_labels_and_expression = False
# change to true if you want to prepare the labels and expression data as well
if preprocess_labels_and_expression:
  methyl_labels = pd.concat([methyl_brca, methyl_luad], axis=1, join="inner")
  methyl_labels.to_csv("/content/drive/MyDrive/MSC/CH3_Project/res/labels_methyl.csv")
  print(expression_brca.index)
  print(expression_luad.index)
  print(len(list(set(expression_brca.index) & set(expression_luad.index))))
  print(len(list(set(expression_brca.columns) & set(expression_luad.columns))))
  expression = pd.concat([expression_brca, expression_luad], axis=1, join="inner")
  expression.to_csv("/content/drive/MyDrive/MSC/CH3_Project/res/expression.csv")

In [ ]:
class Conf:

    dir_hg19 = "/content/drive/MyDrive/MSC/CH3_Project/res/hg_chromosoms"
    checkpoint_dir = ''
    numSurrounding = 400  # per side of CpG i.e. total is x2
    chrArr = [str(i) for i in range(1,23)]
    chrArr.extend(['X', 'Y'])
    suffix = ''

    ### YOUR SETTINGS - START ###
    filename_sequence = 'probeToOneHotAll.csv'
    filename_expression = 'TCGA_E_final_transposed.csv'
    filename_dist = 'distances.csv'
    filename_labels = 'TCGA_CH3_final.csv'  # lessDecimals.csv'

    validation_portion_subjects = 0.1
    validation_portion_probes = 0.1
    train_portion_probes = 0.7

    ### YOUR SETTINGS - END ###

    # Below conf files are intended for use ONLY in dataProcessor, not in model code
    probeToSurroundingSeqFilePrefixAll = '/content/drive/MyDrive/MSC/CH3_Project/res/hg_chromosoms/probeToSurroundingSeqFilePrefixAll/probe_to_surroundingSeq'
    probeToSurroundingSeqFilePrefixChr = '/content/drive/MyDrive/MSC/CH3_Project/res/hg_chromosoms/probeToSurroundingSeqFilePrefixChr/probe_to_surroundingSeq'

    probeToOneHotMtrxFilePrefixChr = '../res/probeToOneHotMtrx_'

    probeToOneHotMtrxFilePrefixAll = '../res/probeToOneHotMtrxAll'+str(suffix)
    probeToOneHotPrefixAll = '../res/probeToOneHotAll'+str(suffix)
    probeToOneHotPrefixChr = '../res/probeToOneHotChr_'+str(suffix)
    numBases = 5
    dfDistances = '../res/distances.csv'
    dfMethylName = 'combined_CH3'
    dfMethyl_BRCA = '/content/drive/MyDrive/MSC/CH3_Project/res/BRCA_methyl.csv'
    dfMethyl_normal = '/content/drive/MyDrive/MSC/CH3_Project/res/Normal_methyl.csv'
    dfExpression_BRCA = '/content/drive/MyDrive/MSC/CH3_Project/res/BRCA_expression.csv'
    dfExpression_normal = '/content/drive/MyDrive/MSC/CH3_Project/res/Normal_expression.csv'

    numSampleInputCpgs = 4
    numInputCpgs = 5000

    epochs = 2
    batch_size = 32
    num_steps = 50000


# Creating distances file

In [ ]:
entries = ["/content/drive/MyDrive/MSC/CH3_Project/res/genomic.gbff"]
import csv

def createGenePositionsDict(gene_to_pos_csv_path="", genes_in_data = []):
    genesInData = genes_in_data
    geneToPos = {}
    from Bio import SeqIO
    # get the gene's pos and insert into dict.
    counter = 0
    for index, filename in enumerate(entries):
        records = SeqIO.parse(filename,"genbank")
        for record in records:
            for fg in record.features:
                if "gene" in fg.qualifiers:
                    genes = fg.qualifiers['gene']
                    for gene in genes:
                        if str(gene).upper() in genesInData:
                            start = fg.location.start.position
                            end = fg.location.end.position
                            for f in record.features:
                                try:
                                    chromosome = f.qualifiers['chromosome'][0]
                                except:
                                    continue
                            geneToPos[gene] = [chromosome, start, end]
                            counter+=1
                            print("\n"+gene+" "+str(chromosome)+"_"+str(start)+"_"+str(end))

    with open(gene_to_pos_csv_path, "w") as outfile:
        writer = csv.writer(outfile)
        genes_list = list(geneToPos.keys())
        columns = ['gene', 'chromosome', 'start', 'end']
        writer.writerow(columns)
        for i, gene in enumerate(genes_list):
            row = [gene] + list(geneToPos[gene])
            writer.writerow(row)

    return geneToPos



def createProbePositionsDict_adjusted(methyl_path, prob_to_pos_csv_path, probes_in_data):
    prob_to_pos = pd.read_csv(methyl_path, header=0, sep='\t', dtype={'Chr': object})
    mask = prob_to_pos['probe'].isin(probes_in_data)
    prob_to_pos = prob_to_pos[mask]
    prob_to_pos = prob_to_pos.drop(columns=['chromHMM'])
    prob_to_pos['chr'] = [c[3:] for c in prob_to_pos['chr']]
    prob_to_pos = prob_to_pos.reindex(['probe', 'start', 'end', 'chr'], axis=1)
    prob_to_pos.to_csv(prob_to_pos_csv_path)
    return prob_to_pos

In [ ]:
def createDistanceMatrx_adjusted(geneToPos, probeToPos, sort_probes, numProbes=-1, preSelectedProbes=False,
                                 genesInCols=False, useInverseDist=False, window_limit=-1, distance_path = "/content/drive/MyDrive/MSC/CH3_Project/res/distances.csv"):
    """
    Creates distances.csv (between gene expression and CpG.
    """
    import math
    probesInData = probeToPos['probe'].unique()
    genesInData = geneToPos['gene'].unique()
    cols = ['Probe']
    # data = []
    genesKept = []
    probesKept = []
    dataToAppend = False
    probeCounter = 0
    first_row_being_added = True
    try:
        os.remove(distance_path)
        print("distances file was removed")
    except:
        print("distances file not removed because wasn't found")
    if numProbes != -1:
        chosenProbes = random.sample(list(probesInData), numProbes)
    else:
        if preSelectedProbes!= False:
          chosenProbes = preSelectedProbes
          print(f"the chosen probes are preselected: \n {chosenProbes}")
        else:
          chosenProbes = probesInData
    if sort_probes:
        chosenProbes.sort()  # sort only if later we want that in dataProcessor we will only have to sort the CH3 file and not the distances file
    with open(distance_path, 'a') as dist_file:
        print(chosenProbes)
        for probe in chosenProbes:
            # print(f"probe {probe}")
            if numProbes != -1:
                # print(f"numProbes != -1")
                if probeCounter >= numProbes:
                    break
            try:
                probesKept.append(probe)
                dataRow = [probe]
                row_has_pos_dist_in_window = False
            except:
                continue
            # print("new genes cycle")
            for gene in genesInData:
                try:
                    gene_start, gene_end = geneToPos[geneToPos["gene"] == gene]["start"], geneToPos[geneToPos["gene"] == gene]["end"]
                    cpg_start, cpg_end = probeToPos[probeToPos["probe"] == probe]["start"], probeToPos[probeToPos["probe"] == probe]["end"]
                    probeChr = probeToPos[probeToPos["probe"] == probe]['chr'].iloc[0]
                    geneChr = geneToPos[geneToPos["gene"] == gene]['chromosome'].iloc[0]
                    if str(probeChr) == str(geneChr):
                        if useInverseDist:
                            distance = 1/float(int(gene_start) - int(cpg_start))
                            if window_limit != -1:
                                if abs(1 / float(distance)) <= window_limit:
                                    row_has_pos_dist_in_window = True

                        else:
                            distance = int(gene_start) - int(cpg_start)
                        dataRow.append(distance)
                    else:
                        if useInverseDist:
                            dataRow.append(0)  # the length of chr1 which is the largest possible distance
                        else:
                            dataRow.append(248956422) # the length of chr1 which is the largest possible distance
                    if probeCounter == 0: # only on the first iteration over all genes add them as column names
                        cols.append(gene)
                        genesKept.append(gene)
                except:
                    continue

            if useInverseDist and window_limit != -1:
                if row_has_pos_dist_in_window:
                    if first_row_being_added:
                        print("first_row_being_added")
                        dist_file.write(','.join(cols))
                        first_row_being_added = False
                    dist_file.write('\n')
                    dist_file.write(','.join(map(str, dataRow)))
                probeCounter += 1
            else:
                if probeCounter == 0:
                    dist_file.write(','.join(cols))
                dist_file.write('\n')
                dist_file.write(','.join(map(str, dataRow)))
                probeCounter += 1



In [ ]:
gene_to_pose_probe_to_pose = False
if gene_to_pose_probe_to_pose:
  genes_braca = list(expression_brca.index.unique())
  genes_luad = list(expression_luad.index.unique())
  genes_in_data = list(set(genes_braca + genes_luad))
  print(f' genes_in_data length {len(genes_in_data)}')

  probes_braca = list(methyl_brca.index)
  probes_luad = list(methyl_luad.index)
  probes_in_data = list(set(probes_luad + probes_braca))
  print(f' probes_in_data length {len(probes_in_data)}')

  # creating a mapping file between each gene in the data, to it's position in the genome (using  genomic.gbff file)
  createGenePositionsDict(gene_to_pos_csv_path = "/content/drive/MyDrive/MSC/CH3_Project/for_distances/geneToPos.csv", genes_in_data = genes_in_data)
  # creating a mapping file between each probe (CpG) in the data, to it's position in the methyl data file 450k_probes_ChroMM.bed
  createProbePositionsDict_adjusted(methyl_path="/content/drive/MyDrive/MSC/CH3_Project/res/450k_probes_ChroMM.bed",
                                                  prob_to_pos_csv_path = "/content/drive/MyDrive/MSC/CH3_Project/for_distances/prob_to_pos.csv",
                                                  probes_in_data = probes_in_data)
else:
  gene_to_pos = pd.read_csv("/content/drive/MyDrive/MSC/CH3_Project/for_distances/geneToPos.csv")
  prob_to_pos = pd.read_csv("/content/drive/MyDrive/MSC/CH3_Project/for_distances/prob_to_pos.csv")

<ipython-input-9-837f019c97db>:21: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  prob_to_pos = pd.read_csv("/content/drive/MyDrive/MSC/CH3_Project/for_distances/prob_to_pos.csv")


In [ ]:
labels_df_sampled = pd.read_csv("/content/drive/MyDrive/MSC/CH3_Project/res/labels_sampled.csv")
cpgs = list(labels_df_sampled['Probe'])

**Creating distances file:**

In [ ]:
distance_path = "/content/drive/MyDrive/MSC/CH3_Project/res/distances_full_2000_base_pairs_around_cpg.csv"
createDistanceMatrx_adjusted(geneToPos=gene_to_pos, probeToPos=prob_to_pos,
                                numProbes=-1, sort_probes=True, preSelectedProbes=False, useInverseDist=True, window_limit=2000, distance_path = distance_path)

**Reading the distance file for validation**

In [ ]:
distances_df = pd.read_csv(distance_path)
distances_df